## Importing Depedencies

In [1]:
# Import dependencies
import regex as re 
import requests
import json
import pandas as pd
from dotenv import load_dotenv
import os
import numpy as np
import time

# <font color='blue'>Part 1: Accessing the New York Times API<font>

#### Accessing api key

In [2]:
load_dotenv()
nyt_api_key = os.getenv('NYT_API_KEY')

#### Building query

In [3]:
#Base url
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

#Filter
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

#Sorting order
sort = 'newest'

#Return fields
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

#Dates
begin_date = "20130101"
end_date = "20230531"

#Query
query_url = (f'{url}api-key={nyt_api_key}&begin_date{begin_date}&end_date{end_date}'+
            f'&fq={filter_query}&sort={sort}&fl={field_list}')

#### Executing query

In [4]:
reviews_list = []
for page in range(20):
    try:
        reviews = requests.get(query_url+f'&page={page}').json()
        reviews_list.append(reviews["response"]["docs"])
        print(f'Page number:{page}')
    except:
        print(f'Page number:{page} did not have a result')
    
    time.sleep(12)


Page number:0
Page number:1
Page number:2
Page number:3
Page number:4
Page number:5
Page number:6
Page number:7
Page number:8
Page number:9
Page number:10
Page number:11
Page number:12
Page number:13
Page number:14
Page number:15
Page number:16
Page number:17
Page number:18
Page number:19


#### Printing the first 5 results in the list

In [5]:
for i in range(5):
    print(f'Review : {i}\n')
    print(json.dumps(reviews_list[i],indent= 4))

Review : 0

[
    {
        "web_url": "https://www.nytimes.com/2024/04/25/movies/unsung-hero-review-christian-pop.html",
        "snippet": "In fact, there\u2019s a lot of singing in the clan whose members inspired this movie and who have racked up five Grammy Awards for their Christian recordings.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018Unsung Hero\u2019 Review: Music Dedicated to the One They Love",
            "kicker": null,
            "content_kicker": null,
            "print_headline": null,
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "subject",
                "value": "Music",
                "rank": 2,
                "major": "N"
            },
            {
            

#### Normailized reviews data frame

In [6]:
normalized_reviews = [pd.json_normalize(item) for item in reviews_list]
reviews_df = pd.concat(normalized_reviews,ignore_index = True)
reviews_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2024/04/25/movies/unsu...,"In fact, there’s a lot of singing in the clan ...",The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2024-04-25T09:04:35+0000,274,‘Unsung Hero’ Review: Music Dedicated to the O...,None,None,None,None,None,None,By Nicolas Rapold,"[{'firstname': 'Nicolas', 'middlename': None, ...",None
1,https://www.nytimes.com/2024/03/07/movies/love...,"In this neo-noir, the ever reliable, always wa...",The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2024-03-07T10:03:26+0000,986,"‘Love Lies Bleeding’ Review: Kristen Stewart, ...",Critic’s Pick,None,"Bad Romance, Hard-Boiled Style",None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
2,https://www.nytimes.com/2024/02/14/movies/blee...,Ewan McGregor plays father to his real-life da...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2024-02-14T23:21:32+0000,406,‘Bleeding Love’ Review: On the Road With Dad,None,None,"Riding Shotgun With Dad, Onscreen and in Reality",None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
3,https://www.nytimes.com/2024/02/14/movies/bob-...,This patchy biopic lauds the Marley of dormito...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2024-02-14T10:02:25+0000,926,‘Bob Marley: One Love’ Review: Mostly Positive...,None,None,"Symbol of Unity, Man of Complexity",None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None
4,https://www.nytimes.com/2024/02/08/movies/moll...,This rom-com brings futuristic absurdity and n...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2024-02-08T12:00:09+0000,275,"‘Molli and Max in the Future’ Review: Love, In...",Critic’s Pick,None,Molli and Max In the Future,None,None,None,By Nicolas Rapold,"[{'firstname': 'Nicolas', 'middlename': None, ...",None


### Extracting movie title

#### Defining the get_title function

In [7]:
def get_title(data):
    """
        Retrieves the movie titles based on regex patterns
        Args:
        data (string): string to be parsed
        Returns:
        string: movie title, if its not found np.nan.
    """
    pattern_1 = r'\D(.*)\D\sReview'
    pattern_2 =r'Review:\s?[In|On]?\s+\W?(.+?)\W+\s'
    if re.findall(pattern_1,data):
        return re.findall(pattern_1,data)[0]
    elif re.findall(pattern_2,data):
        return re.findall(pattern_2,data)[0]
    else:
        return np.nan

#### Applying get_title

In [8]:
reviews_df['Title'] = reviews_df['headline.main'].apply(get_title)

#### Na values After application

In [9]:
reviews_df[reviews_df['Title'].isna()]

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,Title
178,https://www.nytimes.com/2018/05/24/movies/how-...,John Cameron Mitchell’s loony romantic comedy ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2018-05-24T11:00:01+0000,458,Review: An Alien Teaches a Teenage Punk ‘How t...,None,None,Otherworldly Love in a Punk-Rock World,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,NaN
184,https://www.nytimes.com/2018/03/29/movies/outs...,Edie Falco and Jay Duplass connect romanticall...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2018-03-29T11:00:01+0000,422,Review: On Parole and in Love in ‘Outside In’,None,None,On Parole and in Love,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,NaN
187,https://www.nytimes.com/2018/02/15/movies/the-...,Sally Potter directs a very fine cast in a wou...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2018-02-15T12:00:25+0000,598,Review: A London Night Goes Wrong in ‘The Party’,None,None,"Invite Company. Because After All, Misery Lo...",None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,NaN


#### <font color='blue'>Note:<font><b>
**After applying the get_title function the Na values are narrowed down to 3 wich allows us to manually examine them:**<b>

**1) Index 177: 'Review: An Alien Teaches a Teenage Punk ‘How to Talk to Girls at Parties’'** 
    <font color='blue'>As we can see there is not a discernable title here<font>.<b>

**2) index 183: 'Review: On Parole and in Love in ‘Outside In’'**
    <font color='blue'>It is likely due to the structure of the text that the title of the movie is 'Outside In'<font>.<b>

**3) index 186: 'Review: A London Night Goes Wrong in ‘The Party’'**
    <font color='blue'>Like the text above it looks like the title of the movie is ‘The Party’ <font>.<b>



#### Adjusting the Na values

In [10]:
reviews_df['Title'].iloc[183] = 'Outside In'
reviews_df['Title'].iloc[186] = 'The Party'

C:\Users\Chris\AppData\Local\Temp\ipykernel_11368\2754888962.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_df['Title'].iloc[183] = 'Outside In'
C:\Users\Chris\AppData\Local\Temp\ipykernel_11368\2754888962.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_df['Title'].iloc[186] = 'The Party'


#### <font color='blue'>Note:<font><b>
**After examining the data we are left with one Na value (index 177) which does not have any apparent movie title representing one entry out of 200 which should not have a significant impact on our analysis, thus it will be dropped**

In [11]:
reviews_df.drop([177],inplace = True)

#### Generating the title list

In [12]:
titles = reviews_df['Title'].to_list()
titles[:5]

['Unsung Hero',
 'Love Lies Bleeding',
 'Bleeding Love',
 'Bob Marley: One Love',
 'Molli and Max in the Future']

# /////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

# <font color='blue'>Part 2: Accessing The Movie Database API<font>

In [13]:
tmdb_movies_list = []
missing_titles = []
missing_ids = []
raw_details = []
request_counter = 1

#### Accessing API key

In [14]:
load_dotenv()
tmdb_api_key = os.getenv('TMDB_API_KEY')

#### Building Query

In [15]:
#Base url
url = 'https://api.themoviedb.org/3/search/movie?query='
details_url = 'https://api.themoviedb.org/3/movie/'

#### Defining retrieving information functions

#### get_nested_info

In [16]:
def get_nested_info(details,key):

    """
    Generates a list from the nested items in the details dictionary
    Args:
        details (dict): Dictionary with nested items.
        key (string): key for the information to be extracted.
    Returns:
        list: list of nested items.
    """
    sub_list = []
    items = details[key] 
    for item in items:
        sub_list.append(item['name'])
    return sub_list

#### get_movie_details

In [17]:
def get_movie_details(details):
    """
    Generates a dictionary from the subset of keys of the original dictionary.
    Args:
        details (dict): Dictionary of items to be extracted.
    Returns:
        dict: Dictionary of retrieved information
    """
    movie_dict = {}
    details_list = ['title','original_title','budget','original_language',
                    'homepage','overview','runtime','revenue',
                    'release_date','vote_average','vote_count']
    
    keys = ['genres','spoken_languages','production_countries']
    
    #Retrieving items
    for detail in details_list:
        
        movie_dict[detail] = details[detail]
    
    #Retreiving nested items
    for key in keys:
        movie_dict[key] = get_nested_info(details,key)

    return movie_dict

#### Extracting information from the TMDB

In [18]:
for title in titles:
    movie_query_url = (f'{url}{title}&api_key={tmdb_api_key}')
    try:
        response = requests.get(movie_query_url).json()

         #Querying movie title info
        id = str(response['results'][0]['id'])
        id_query = (f'{details_url}{id}?&api_key={tmdb_api_key}')

        #Querying movie details info
        details = requests.get(id_query).json()
        raw_details.append(details)
        
        movie_dict = get_movie_details(details)
        tmdb_movies_list.append(movie_dict)

        #Movie title found
        print(f'Title:{title} found')
        
        
    except:
        print(f'Movie title:{title} not found')
        missing_titles.append(title)
        missing_ids.append(id)
    request_counter+=1
    if request_counter%50 == 0:
        time.sleep(1)
        print('Loop sleeping')
        

Title:Unsung Hero found
Title:Love Lies Bleeding found
Title:Bleeding Love found
Title:Bob Marley: One Love found
Title:Molli and Max in the Future found
Title:The Taste of Things found
Title:Pictures of Ghosts found
Title:Anyone but You found
Title:Fallen Leaves found
Title:Radical found
Title:Fingernails found
Title:My Love Affair With Marriage found
Title:Foe found
Title:The Creator found
Title:My Sailor, My Love found
Title:Cassandro found
Title:Love at First Sight found
Title:We Kill for Love found
Title:Choose Love found
Title:Before, Now & Then found
Title:Love Life found
Title:Red, White & Royal Blue found
Title:Love in Taipei found
Title:The Eternal Memory found
Title:Passages found
Title:Sublime found
Title:Shooting Stars found
Title:The Attachment Diaries found
Title:What’s Love Got to Do With It found
Title:You Can Live Forever found
Title:A Tourist’s Guide to Love found
Title:Other People’s Children found
Title:One True Loves found
Title:The Lost Weekend: A Love Story foun

#### List of missing titles and there movie Ids

In [19]:
missing_titles

['Aulcie',
 'In ‘Tea With the Dames',
 'In ‘Rogers Park',
 'If You Love Godard',
 'In ‘Forever My Girl',
 'In ‘Love Beats Rhymes']

#### reviews_df subset for the missing titles

In [20]:
reviews_df[reviews_df['Title'].isin(missing_titles)]

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,Title
76,https://www.nytimes.com/2021/11/16/movies/aulc...,This melodramatic documentary chronicles how A...,The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2021-11-16T21:35:40+0000,259,"‘Aulcie’ Review: Love and Basketball, in Israel",None,None,Aulcie,None,None,None,By Natalia Winkelman,"[{'firstname': 'Natalia', 'middlename': None, ...",None,Aulcie
174,https://www.nytimes.com/2018/09/20/movies/tea-...,This documentary with a simple premise — Eilee...,The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2018-09-20T11:00:01+0000,271,"Review: In ‘Tea With the Dames,’ Four Legends ...",None,None,Tea With the Dames,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,In ‘Tea With the Dames
179,https://www.nytimes.com/2018/04/26/movies/revi...,Two couples struggle to find serenity as middl...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2018-04-26T11:00:10+0000,325,"Review: In ‘Rogers Park,’ Life and Love in a C...",None,None,Rogers Park,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,In ‘Rogers Park
181,https://www.nytimes.com/2018/04/18/movies/goda...,A portrait of a filmmaker in crisis that might...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2018-04-18T23:02:46+0000,937,"Review: If You Love Godard, You’ll Hate-Watch ...",None,None,"A Portrait and, for Some, a Hate-Watch",None,None,None,By A.O. Scott,"[{'firstname': 'A.', 'middlename': 'O.', 'last...",None,If You Love Godard
189,https://www.nytimes.com/2018/01/19/movies/fore...,There are few hard feelings in Bethany Ashton ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2018-01-19T17:22:06+0000,273,"Review: In ‘Forever My Girl,’ a Romance Resume...",None,None,Hitting Pause On Love,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,In ‘Forever My Girl
195,https://www.nytimes.com/2017/11/30/movies/love...,The rap luminary RZA and the hip-hop musician ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2017-11-30T12:00:05+0000,319,"Review: In ‘Love Beats Rhymes,’ a Hip-Hop Arti...",None,None,Love Beats Rhymes,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,In ‘Love Beats Rhymes


#### tmdb_movies_list sample

In [21]:
tmdb_movies_list[:3]

[{'title': 'Unsung Hero',
  'original_title': 'Unsung Hero',
  'budget': 0,
  'original_language': 'en',
  'homepage': 'https://unsunghero.movie/',
  'overview': "When David Smallbone's successful music company collapses, he moves his family from Australia to the United States in search of a brighter future. With nothing more than their six children, their suitcases, and their love of music, David and his pregnant wife Helen set out to rebuild their lives from the ground up. Based on a remarkable true story, a mum's faith stands against all odds; and inspires her husband and children to hold onto theirs.",
  'runtime': 112,
  'revenue': 0,
  'release_date': '2024-04-26',
  'vote_average': 10.0,
  'vote_count': 1,
  'genres': ['Drama'],
  'spoken_languages': ['English'],
  'production_countries': ['United States of America']},
 {'title': 'Love Lies Bleeding',
  'original_title': 'Love Lies Bleeding',
  'budget': 0,
  'original_language': 'en',
  'homepage': 'https://tickets.loveliesblee

#### Generating the TMDB dataframe

In [22]:
tmdb_df = pd.DataFrame(tmdb_movies_list)
tmdb_df.head()

,title,original_title,budget,original_language,homepage,overview,runtime,revenue,release_date,vote_average,vote_count,genres,spoken_languages,production_countries
0,Unsung Hero,Unsung Hero,0,en,https://unsunghero.movie/,When David Smallbone's successful music compan...,112,0,2024-04-26,10.000,1,[Drama],[English],[United States of America]
1,Love Lies Bleeding,Love Lies Bleeding,0,en,https://tickets.loveliesbleeding.movie,Reclusive gym manager Lou falls hard for Jacki...,104,8000000,2024-03-08,7.488,41,"[Crime, Romance, Thriller]",[English],"[United Kingdom, United States of America]"
2,Love Lies Bleeding,Love Lies Bleeding,0,en,https://tickets.loveliesbleeding.movie,Reclusive gym manager Lou falls hard for Jacki...,104,8000000,2024-03-08,7.488,41,"[Crime, Romance, Thriller]",[English],"[United Kingdom, United States of America]"
3,Bob Marley: One Love,Bob Marley: One Love,70000000,en,https://www.onelovemovie.com,Jamaican singer-songwriter Bob Marley overcome...,104,176527503,2024-02-14,7.000,499,"[Music, History, Drama]","[English, Français]",[United States of America]
4,Molli and Max in the Future,Molli and Max in the Future,0,en,http://www.michaellitwak.com/molli-max-in-the-...,A sci-fi romantic comedy about a man and woman...,93,0,2024-02-09,8.667,3,"[Comedy, Romance, Science Fiction]",[English],[United States of America]


# ////////////////////////////////////////////////////////////////////////////

# <font color='blue'>Part 3: Merging and Cleaning the Data for Export<font>

#### <font color='blue'>Note:<font>
**The data frames were merged on 'left' merge with tmdb on the left in order to removed the missing titles from the tmdb query.**

#### Reviews data frame shape

In [23]:
reviews_df.shape

(199, 17)

#### TMDB data frame shape

In [24]:
tmdb_df.shape

(193, 14)

#### Renaming the tmdb 'title' to 'Title' to match reviews_df 'Title' column

In [25]:
tmdb_df.rename(columns = {'title':'Title'},inplace = True)

#### Data frame merge

In [26]:
merged_df = tmdb_df.merge(reviews_df,how='left',on='Title')
merged_df.head()

,Title,original_title,budget,original_language,homepage,overview,runtime,revenue,release_date,vote_average,...,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,Unsung Hero,Unsung Hero,0,en,https://unsunghero.movie/,When David Smallbone's successful music compan...,112,0,2024-04-26,10.000,...,‘Unsung Hero’ Review: Music Dedicated to the O...,None,None,None,None,None,None,By Nicolas Rapold,"[{'firstname': 'Nicolas', 'middlename': None, ...",None
1,Love Lies Bleeding,Love Lies Bleeding,0,en,https://tickets.loveliesbleeding.movie,Reclusive gym manager Lou falls hard for Jacki...,104,8000000,2024-03-08,7.488,...,"‘Love Lies Bleeding’ Review: Kristen Stewart, ...",Critic’s Pick,None,"Bad Romance, Hard-Boiled Style",None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
2,Love Lies Bleeding,Love Lies Bleeding,0,en,https://tickets.loveliesbleeding.movie,Reclusive gym manager Lou falls hard for Jacki...,104,8000000,2024-03-08,7.488,...,"‘Love Lies Bleeding’ Review: Kristen Stewart, ...",Critic’s Pick,None,"Bad Romance, Hard-Boiled Style",None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None
3,Bob Marley: One Love,Bob Marley: One Love,70000000,en,https://www.onelovemovie.com,Jamaican singer-songwriter Bob Marley overcome...,104,176527503,2024-02-14,7.000,...,‘Bob Marley: One Love’ Review: Mostly Positive...,None,None,"Symbol of Unity, Man of Complexity",None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None
4,Molli and Max in the Future,Molli and Max in the Future,0,en,http://www.michaellitwak.com/molli-max-in-the-...,A sci-fi romantic comedy about a man and woman...,93,0,2024-02-09,8.667,...,"‘Molli and Max in the Future’ Review: Love, In...",Critic’s Pick,None,Molli and Max In the Future,None,None,None,By Nicolas Rapold,"[{'firstname': 'Nicolas', 'middlename': None, ...",None


#### Merged data frame information summary

In [27]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193 entries, 0 to 192
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Title                    193 non-null    object 
 1   original_title           193 non-null    object 
 2   budget                   193 non-null    int64  
 3   original_language        193 non-null    object 
 4   homepage                 193 non-null    object 
 5   overview                 193 non-null    object 
 6   runtime                  193 non-null    int64  
 7   revenue                  193 non-null    int64  
 8   release_date             193 non-null    object 
 9   vote_average             193 non-null    float64
 10  vote_count               193 non-null    int64  
 11  genres                   193 non-null    object 
 12  spoken_languages         193 non-null    object 
 13  production_countries     193 non-null    object 
 14  web_url                  1

#### Subset of the 'genres', 'spoken_languages', and 'production_countries' columns

In [28]:
merged_df[['genres','spoken_languages','production_countries']].head()

,genres,spoken_languages,production_countries
0,[Drama],[English],[United States of America]
1,"[Crime, Romance, Thriller]",[English],"[United Kingdom, United States of America]"
2,"[Crime, Romance, Thriller]",[English],"[United Kingdom, United States of America]"
3,"[Music, History, Drama]","[English, Français]",[United States of America]
4,"[Comedy, Romance, Science Fiction]",[English],[United States of America]


#### Casting the subset as string

In [29]:
merged_df['genres'] = merged_df['genres'].astype('str')
merged_df['spoken_languages'] = merged_df['spoken_languages'].astype('str')
merged_df['production_countries'] = merged_df['production_countries'].astype('str')

#### Removing unwanted characters [ ] '

In [30]:
merged_df['genres'] = merged_df['genres'].str.replace(r"[\[\]\']",'',regex=True)
merged_df['spoken_languages'] = merged_df['spoken_languages'].str.replace(r"[\[\]\']",'',regex=True)
merged_df['production_countries'] = merged_df['production_countries'].str.replace(r"[\[\]\']",'',regex=True)

#### Merged data frame sample

In [31]:
merged_df[['genres','spoken_languages','production_countries']].head()

,genres,spoken_languages,production_countries
0,Drama,English,United States of America
1,"Crime, Romance, Thriller",English,"United Kingdom, United States of America"
2,"Crime, Romance, Thriller",English,"United Kingdom, United States of America"
3,"Music, History, Drama","English, Français",United States of America
4,"Comedy, Romance, Science Fiction",English,United States of America


#### Saving data frame to csv

In [32]:
merged_df.to_csv('merged_df.csv',index=False)